In [ ]:
!pip install accelerate bitsandbytes transformers peft datasets trl

In [ ]:
!pip install -U sentence-transformers transformers

In [ ]:
import pandas as pd
df = pd.read_csv('parsing.csv')
documents = df.apply(lambda row: f"Q: {row['Question']}\nA: {row['Answer']}", axis=1).tolist()


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2", cache_folder="downloader")
embeddings = model.encode(documents, convert_to_tensor=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.6 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings.cpu()))
# Save FAISS index
faiss.write_index(index, "index.faiss")

# Save your documents
with open("context_docs.txt", "w") as f:
    f.write("\n---\n".join(documents))

# Later, reload like this:
index = faiss.read_index("index.faiss")
with open("context_docs.txt") as f:
    documents = f.read().split("\n---\n")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig
model_id = "Norquinal/Mistral-7B-claude-chat"
tokenizer = AutoTokenizer.from_pretrained("Norquinal/Mistral-7B-claude-chat", cache_dir="downloader")
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16, offload_folder="/offload", cache_dir="downloader")

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def generate_answer(query, index, documents, embed_model, lm_model, tokenizer, max_tokens=300):
    import numpy as np
    import time

    # Step 1: Embed the query
    query_embedding = embed_model.encode(query)

    # Step 2: Retrieve relevant chunks
    D, I = index.search(np.array([query_embedding]), k=5)

    # Step 3: Clean retrieved chunks to strip Q: and A:
    def strip_qa_noise(text):
        lines = text.split("\n")
        return "\n".join(line for line in lines if not line.strip().startswith(("Q:", "A:", "Question:", "Answer:")))

    retrieved = [strip_qa_noise(documents[i]) for i in I[0]]
    context = "\n".join(retrieved)

    # Step 4: Format prompt with clear instructions
    prompt = f"""You are a helpful technical assistant.

Use only the following context to answer the current question. Do not repeat unrelated prior questions.

Context:
{context}

Question: {query}
Answer:"""

    # Step 5: Tokenize and generate
    device = next(lm_model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    start = time.time()
    output = lm_model.generate(**inputs, max_new_tokens=max_tokens)


    return tokenizer.decode(output[0], skip_special_tokens=True)



generate_answer(
    query="What should be done if the edge/cluster fails to come up after a management interface toggle",
    index=index,
    documents=documents,
    embed_model=SentenceTransformer("all-MiniLM-L6-v2"),
    lm_model=model,
    tokenizer=tokenizer
)



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
!cp index.faiss /content/drive/MyDrive/rag/index.faiss


In [ ]:
!cp context_docs.txt /content/drive/MyDrive/rag/context_docs.txt


In [ ]:
!cp -r downloader /content/drive/MyDrive/rag/downloader

